# Homework 1: Perceptrons
### Due Thursday, January 17, 2019, 5pm
#### Jesse Zhu
#### ML-W2019

Using the MNIST hand-written digits dataset, we aim to use perceptrons to categorize the labeled data through supervised learning. As this is a single-layer, the result is not expected to be perfect, but should be noticeably better than random. Each data point will be 785 values between 0 and 1, representing the grayscale value of each pixel in the original image. These values will be used to update our weight values over the course of 50 epochs, resulting in our 10 perceptrons (digits 0 to 9) being trained at the end. Finally, we will run test data through these trained perceptrons and take the maximum value of the 10 perceptrons as the predicted result.

In [70]:
#import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [30]:
#IMPORT DATA#################
testfile = "mnist_test.csv"
trainfile = "mnist_train.csv"

# test_data = pd.read_csv(testfile)
# train_data = pd.read_csv(trainfile)

#Data is 785 columns by N rows. First column = Label (0-9), others = 0:255
test_data = np.genfromtxt(testfile, skip_header=True, delimiter=',')
train_data = np.genfromtxt(trainfile, skip_header=True, delimiter=',')

In [46]:
#PREPROCESSING###############

#SCALE by 255
test_data[:,1:] /= 255
train_data[:,1:] /= 255
print((test_data[0,:]))

[2.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.45490196
 0.49019608 0.67058824 1.         1.         0.5

In [56]:
#SHUFFLE (not required this assignment)

#print((test_data[0,:]))

#INITIALIZE RANDOM WEIGHTS
#785,10
init_weights = (np.random.rand(785,10) * 0.10) - 0.05

learning_rates = [0.01, 0.1, 1.0]

[ 0.04631771  0.00114123  0.00894236  0.04313398  0.04093737  0.02829112
  0.00319776 -0.03990701  0.03173384  0.00164862]


In [79]:
#Accuraccy function
def acc(weights, data, debug = 0):
    """input: 
        weights: matrix of 785 weights by 10 classes, where the first weight is the bias
        data: matrix of N data points by 785 values, where the first value is the target (0-9), and the rest are
            scaled gray-scale values between 0 and 1
        output: Percentage of correct classifications, where a classification is taken using the highest perceptron
            output value
    """
    ret = 0
    length = len(data[:,0])
    out = data[:, 1:] @ weights[1:, :]
    print(out.shape)
    amax = out.argmax(axis=1)
    print(amax[0:25])
    ret = np.sum(amax == data[:,0])
    return ret/length

#Drawing function
def draw_digit(data):
    """
    input:
        1x785 matrix of grayscaled pixel values
    output: image of (1:785 pixels, skipping first label)
    """
    img = np.reshape(data[1:], (28, 28))
    print(img)
    plt.imshow(img)
    plt.show
    

In [80]:
w = init_weights
print(acc(w, test_data))
#draw_digit(test_data[0, :])

(9999, 10)
[7 1 7 7 1 3 7 4 1 7 1 7 7 7 1 1 1 1 1 1 7 1 7 1 7]
0.07200720072007201


Appendix of Resources
1. Matrix multiplication
https://stackoverflow.com/questions/21562986/numpy-matrix-vector-multiplication

```a = np.random.rand(1,3)
b = np.random.rand(3,5)
print(a@b)

print(np.zeros([3,3]))
```

2. Argmax arrays
https://stackoverflow.com/questions/5469286/how-to-get-the-index-of-a-maximum-element-in-a-numpy-array-along-one-axis
```
mm.argmax(axis=1)
```

3. Plotting 2d arrays
https://stackoverflow.com/questions/16492830/colorplot-of-2d-array-matplotlib

4. Count # Equal Array Elements
https://stackoverflow.com/questions/25490641/check-how-many-elements-are-equal-in-two-numpy-arrays-python
```
np.sum(a == b)
```
